# Week 5: Simple single-cell RNA-seq analysis

1. fetch some single-cell data and create a cell-gene expression matrix and the associated AnnData object;
2. cluster the data to understand the cell structure; and
3. annotate the data with biologically relevant information.

All steps are based on Single-cell Best Practices Book (https://www.sc-best-practices.org/). Feel free to use it! You will still need to automate the process and ensure that it can be run.

## Step 0: Setup

### Install Dependencies

In [1]:
from pathlib import Path
import urllib.request
import subprocess
import gzip
import shutil
import subprocess
import pyroe
import numpy as np
import tarfile
import os
import sys
import time

/home/nick/miniconda3/envs/scrna/lib/python3.10/site-packages/pyranges/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### Set up working directories

In [2]:
notebook_dir = Path.cwd()
data_dir = notebook_dir / "data"
data_dir.mkdir(parents=True, exist_ok=True)
print(f"Working directory: {notebook_dir}")
print(f"Data directory: {data_dir}")

Working directory: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code
Data directory: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data


## Step 1:

Get the sample data here: https://app.box.com/s/lx2xownlrhz3us8496tyu9c4dgade814. This data contains the single-cell FASTQ files, as well as the reference genome (chr5) and the GTF file with transcript information.

The list of whitelist barcodes is available [here](https://github.com/f0t1h/3M-february-2018/raw/refs/heads/master/3M-february-2018.txt.gz).

In [3]:
whitelist_url = "https://raw.githubusercontent.com/f0t1h/3M-february-2018/master/3M-february-2018.txt.gz"
whitelist_gz = data_dir / "3M-february-2018.txt.gz"
whitelist_path = data_dir / "whitelist.txt"

if not whitelist_path.exists():
    print("Downloading 10x whitelist...")
    urllib.request.urlretrieve(whitelist_url, whitelist_gz)
    print("Extracting whitelist...")
    with gzip.open(whitelist_gz, 'rb') as f_in:
        with open(whitelist_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    whitelist_gz.unlink()
    print(f"Whitelist saved to: {whitelist_path}")
else:
    print(f"Whitelist already exists: {whitelist_path}")

Extracting whitelist...
Whitelist saved to: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/whitelist.txt


In [4]:
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)
toy_dataset_gz = data_dir / "toy_read_ref_set.tar.gz"

print("Extracting dataset")

with tarfile.open(toy_dataset_gz, 'r:gz') as tar:
    tar.extractall(path=data_dir)
    
toy_ref_read = data_dir / "toy_ref_read"
fastq_dir = toy_ref_read / "toy_read_fastq"
ref_dir = toy_ref_read / "toy_human_ref"
r1_fastq = fastq_dir / "selected_R1_reads.fastq"
r2_fastq = fastq_dir / "selected_R2_reads.fastq"
genome_fa = ref_dir / "fasta" / "genome.fa"
genes_gtf = ref_dir / "genes" / "genes.gtf"

files_ok = True

for filepath, description in [
    (r1_fastq, "R1 reads"),
    (r2_fastq, "R2 reads"),
    (genome_fa, "Genome FASTA"),
    (genes_gtf, "GTF annotation")
]:
    exists = filepath.exists()
    print(f"{description}: {filepath}")
    if not exists:
        files_ok = False

if files_ok:
    print("Files successfully extracted")
else:
    print("Some files are missing")

Extracting dataset
R1 reads: data/toy_ref_read/toy_read_fastq/selected_R1_reads.fastq
R2 reads: data/toy_ref_read/toy_read_fastq/selected_R2_reads.fastq
Genome FASTA: data/toy_ref_read/toy_human_ref/fasta/genome.fa
GTF annotation: data/toy_ref_read/toy_human_ref/genes/genes.gtf
Files successfully extracted


## Step 2:

Use Alevin-fry to align and quantify this data to the reference genome. See [here](https://www.sc-best-practices.org/introduction/raw_data_processing.html#a-real-world-example) for the installation steps.

In [5]:
#!/usr/bin/env python3



# --- Configuration (edit if needed) ---
notebook_dir = Path.cwd()
data_dir = notebook_dir / "data"
toy_dataset_gz = data_dir / "toy_read_ref_set.tar.gz"   # already extracted by your Step 1
toy_ref_read = data_dir / "toy_ref_read"
fastq_dir = toy_ref_read / "toy_read_fastq"
ref_dir = toy_ref_read / "toy_human_ref"

r1_fastq = fastq_dir / "selected_R1_reads.fastq"
r2_fastq = fastq_dir / "selected_R2_reads.fastq"
genome_fa = ref_dir / "fasta" / "genome.fa"
genes_gtf = ref_dir / "genes" / "genes.gtf"

# whitelist (use canonical raw.githubusercontent URL)
whitelist_url_candidates = [
    "https://raw.githubusercontent.com/f0t1h/3M-february-2018/master/3M-february-2018.txt.gz",
    "https://github.com/f0t1h/3M-february-2018/raw/refs/heads/master/3M-february-2018.txt.gz"
]
whitelist_gz = data_dir / "3M-february-2018.txt.gz"
whitelist_path = data_dir / "whitelist.txt"

# output directories
output_dir = data_dir / "alevin_output"
index_dir = output_dir / "salmon_index"
map_dir = output_dir / "map"
quant_dir = output_dir / "quant"

# salmon/read settings
read_length = 91   # adjust if your reads differ
salmon_threads = "4"
salmon_libtype = "ISR"   # keep as used earlier; change if necessary
chromium_flag = "--chromiumV3"  # use correct chromium flag for your chemistry; adjust if needed

# --- Helpers ---
def run(cmd, env=None, cwd=None, check=True):
    """Run a command, print stdout/stderr, and return completed process."""
    print("Running:", " ".join(cmd))
    proc = subprocess.run(cmd, capture_output=True, text=True, env=env, cwd=cwd)
    if proc.stdout:
        print("STDOUT:", proc.stdout.strip())
    if proc.stderr:
        print("STDERR:", proc.stderr.strip())
    if check and proc.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)} (exit {proc.returncode})")
    return proc

def find_executable(name):
    from shutil import which
    return which(name)

def try_fix_salmon_binary(env):
    """
    If salmon call failed with GLIBC_PRIVATE we try to prefer system /usr/bin if present.
    This sets PATH in env to use /usr/bin first.
    """
    sys_salmon = Path("/usr/bin/salmon")
    if sys_salmon.exists() and os.access(sys_salmon, os.X_OK):
        new_env = dict(os.environ)
        new_env["PATH"] = "/usr/bin:" + new_env.get("PATH", "")
        return new_env
    return env

# --- Preflight checks ---
print("\n=== Step 2: Alevin-fry Alignment and Quantification ===\n")
data_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
index_dir.mkdir(parents=True, exist_ok=True)
map_dir.mkdir(parents=True, exist_ok=True)
quant_dir.mkdir(parents=True, exist_ok=True)

# check input files
missing = []
for p, desc in [(r1_fastq, "R1 FASTQ"), (r2_fastq, "R2 FASTQ"),
                (genome_fa, "Genome FASTA"), (genes_gtf, "GTF")]:
    print(f"{desc}: {p} -> exists={p.exists()}")
    if not p.exists():
        missing.append((p, desc))
if missing:
    print("ERROR: missing required inputs. Aborting.")
    for p, desc in missing:
        print(f" - {desc}: {p}")
    sys.exit(1)

# ensure whitelist present
if not whitelist_path.exists():
    downloaded = False
    for url in whitelist_url_candidates:
        try:
            print(f"Downloading whitelist from {url} ...")
            urllib.request.urlretrieve(url, whitelist_gz)
            with gzip.open(whitelist_gz, "rb") as fin, open(whitelist_path, "wb") as fout:
                shutil.copyfileobj(fin, fout)
            whitelist_gz.unlink(missing_ok=True)
            print("Whitelist saved to:", whitelist_path)
            downloaded = True
            break
        except Exception as e:
            print("Failed to download from", url, ":", e)
            if whitelist_gz.exists():
                whitelist_gz.unlink(missing_ok=True)
    if not downloaded:
        raise RuntimeError("Could not download whitelist from any candidate URL.")
else:
    print("Whitelist already exists:", whitelist_path)

# check tools presence (salmon, alevin-fry)
salmon_path = find_executable("salmon")
alevin_fry_path = find_executable("alevin-fry")

if salmon_path is None:
    raise RuntimeError("salmon not found on PATH. Install salmon (apt or conda) and ensure available.")

if alevin_fry_path is None:
    raise RuntimeError("alevin-fry not found on PATH. Install alevin-fry and ensure available.")

print("Found salmon at:", salmon_path)
print("Found alevin-fry at:", alevin_fry_path)

# check salmon version, and attempt to detect GLIBC-private broken binary
def salmon_version_and_check(env=None):
    try:
        p = subprocess.run(["salmon","--version"], capture_output=True, text=True, env=env)
        stdout = p.stdout.strip()
        stderr = p.stderr.strip()
        return p.returncode, stdout, stderr
    except FileNotFoundError:
        return 1, "", "salmon not found"

rc, out, err = salmon_version_and_check()
if rc != 0 and "GLIBC_PRIVATE" in err:
    print("Detected GLIBC_PRIVATE error from salmon. Attempting to prefer system /usr/bin/salmon if available.")
    env_try = try_fix_salmon_binary(dict(os.environ))
    rc2, out2, err2 = salmon_version_and_check(env=env_try)
    if rc2 == 0:
        print("System salmon appears usable. Will run commands with PATH preferring /usr/bin.")
        run_env = env_try
    else:
        print("System salmon did not fix the issue. Aborting.")
        print("salmon stderr:", err)
        raise RuntimeError("salmon binary is incompatible with this system (GLIBC_PRIVATE). Install a compatible salmon.")
else:
    run_env = None  # use default environment
    print("salmon version OK:", out or err)

# --- Step 2.1: Build splici txome with pyroe (if not exist) ---
splici_fa = index_dir / "splici_fl86.fa"
splici_prefix = "splici"
if not splici_fa.exists():
    print("\n[1/4] Building splici txome with pyroe...")
    try:
        import pyroe
        pyroe.make_splici_txome(
            gtf_path=str(genes_gtf),
            genome_path=str(genome_fa),
            read_length=read_length,
            output_dir=str(index_dir),
            flank_trim_length=5,
            filename_prefix=splici_prefix
        )
    except Exception as e:
        print("pyroe make_splici_txome failed:", e)
        # try data type fix for gtf using pyranges if available
        try:
            import pyranges as pr
            print("Attempting to load GTF with pyranges and correct dtypes...")
            gtf = pr.read_gtf(str(genes_gtf))
            gtf.Start = gtf.Start.astype('int32')
            gtf.End = gtf.End.astype('int32')
            corrected_gtf = index_dir / "corrected_genes.gtf"
            gtf.to_gtf(str(corrected_gtf))
            pyroe.make_splici_txome(
                gtf_path=str(corrected_gtf),
                genome_path=str(genome_fa),
                read_length=read_length,
                output_dir=str(index_dir),
                flank_trim_length=5,
                filename_prefix=splici_prefix
            )
        except Exception as e2:
            print("Fallback pyroe attempt failed:", e2)
            raise

    print("splici txome files should now be in:", index_dir)
else:
    print("\n[1/4] splici txome already exists:", splici_fa)

# --- Step 2.2: Build salmon index (if not exists) ---
salmon_idx_dir = index_dir / "salmon_idx"
if not (salmon_idx_dir.exists() and (salmon_idx_dir / "hash.bin").exists()):
    print("\n[2/4] Building salmon index...")
    salmon_index_cmd = [
        "salmon", "index",
        "-t", str(index_dir / f"{splici_prefix}_fl86.fa"),
        "-i", str(salmon_idx_dir),
        "-p", salmon_threads
    ]
    run(salmon_index_cmd, env=run_env)
    print("Salmon index built at:", salmon_idx_dir)
else:
    print("\n[2/4] Salmon index already exists:", salmon_idx_dir)

# --- Step 2.3: Run salmon alevin (mapping) ---
alevin_map_dir = map_dir / "alevin"
if not alevin_map_dir.exists():
    print("\n[3/4] Running salmon alevin (mapping)...")
    alevin_cmd = [
        "salmon", "alevin",
        "-l", salmon_libtype,
        "-i", str(salmon_idx_dir),
        "-1", str(r1_fastq),
        "-2", str(r2_fastq),
        "-o", str(map_dir),
        "-p", salmon_threads,
        "--sketch"
    ]
    # include chromium flag if requested
    if chromium_flag:
        alevin_cmd.append(chromium_flag)
    run(alevin_cmd, env=run_env)
    print("Mapping (salmon alevin) complete; results in:", map_dir)
else:
    print("\n[3/4] Mapping already exists at:", alevin_map_dir)

# --- Step 2.4: generate-permit-list (alevin-fry) ---
print("\n[4/4] Generating permit list with alevin-fry...")
gen_permit_cmd = [
    "alevin-fry", "generate-permit-list",
    "-i", str(map_dir),
    "-o", str(quant_dir),
    "-u", str(whitelist_path),
    "-d", "fw"   # forward stranded; change to '1' or '2' if your data differs
]
run(gen_permit_cmd, env=run_env)
print("Permit list generated in", quant_dir)

# --- Collate ---
print("\n[5/4] Collating alevin outputs...")
collate_cmd = [
    "alevin-fry", "collate",
    "-r", str(map_dir),
    "-i", str(quant_dir),
    "-t", "4"
]
run(collate_cmd, env=run_env)
print("Collate complete.")

# --- Quant ---
print("\n[6/4] Quantifying with alevin-fry (quant)...")
t2g = index_dir / "splici_fl86_t2g_3col.tsv"
if not t2g.exists():
    # attempt to locate a t2g file created by pyroe
    candidates = list(index_dir.glob("*t2g*.tsv")) + list(index_dir.glob("*t2g*.txt"))
    if candidates:
        t2g_file = candidates[0]
        print("Found t2g file:", t2g_file)
    else:
        raise RuntimeError("t2g mapping file not found in index directory; quant requires a splici->gene t2g file.")

quant_cmd = [
    "alevin-fry", "quant",
    "-i", str(quant_dir),
    "-o", str(quant_dir),
    "-r", "cr-like",
    "-m", str(t2g),
    "-t", "4",
    "--use-mtx"
]
run(quant_cmd, env=run_env)
print("Quantification complete. Final results in:", quant_dir)

print("\nAll done. You can now load the results into Scanpy/AnnData (example):")
print(f"import scanpy as sc\nadata = sc.read_10x_mtx('{quant_dir}', var_names='gene_symbols', cache=True)")



=== Step 2: Alevin-fry Alignment and Quantification ===

R1 FASTQ: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/toy_ref_read/toy_read_fastq/selected_R1_reads.fastq -> exists=True
R2 FASTQ: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/toy_ref_read/toy_read_fastq/selected_R2_reads.fastq -> exists=True
Genome FASTA: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/toy_ref_read/toy_human_ref/fasta/genome.fa -> exists=True
GTF: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/toy_ref_read/toy_human_ref/genes/genes.gtf -> exists=True
Whitelist already exists: /mnt/c/Users/Nick/documents/github/fall25-csc-bioinf/week6/code/data/whitelist.txt
Found salmon at: /home/nick/miniconda3/envs/scrna/bin/salmon
Found alevin-fry at: /home/nick/miniconda3/envs/scrna/bin/alevin-fry
salmon version OK: salmon 1.10.3

[1/4] Building splici txome with pyroe...
splici txome files should now be in: /mnt/c/Users/Nick/documen

## Step 3: 

Perform cell clustering (via Leiden modularity algorithm). Output the clustering plot.

## Step 4:

Perform automatic cell annotation via CellTypist. Annotate the plot with the cell types.

## Time Estimate: 8 Hours